In [1]:
import joblib
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

from IPython.display import display

from sklearn.metrics import accuracy_score, log_loss, roc_auc_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ParameterGrid
from sklearn.model_selection import GridSearchCV

import seaborn as sns



import sympy as sym
from sympy import Symbol, sympify, lambdify, abc, SympifyError

from gplearn.genetic import SymbolicClassifier
from gplearn.genetic import SymbolicRegressor
from sympy import *

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import PolynomialFeatures
import types
import random

import graphviz

In [2]:
joblib.__version__

'1.0.1'

In [3]:
np.random.seed(42)
random.seed(42)

## Load Data

In [4]:
#path = "./data/replica_pump_data.csv"
path = "./data/replica_pump_data.csv"
pump_data_replica = pd.read_csv(path)
print(pump_data_replica.shape)

(26381042, 5)


In [5]:
pump_data_replica.head()

,energy_norm_log,temperature_diff,rms_norm_log,details_ratedhead,state
0,-4.642337,-0.585072,-2.831278,47.369469,1
1,-4.384410,-2.051363,-2.900545,120.240341,1
2,-5.047895,2.104730,-2.742720,92.577971,1
3,-4.962318,0.375291,-2.975236,75.714544,1
4,-5.082340,-1.878716,-2.900094,19.732252,1


## Load Random Forest Model

In [6]:
with open("./data/randForestBest_20201002.pkl", 'rb') as f:
    random_forest_model = joblib.load(f)  

/home/smarton/anaconda3/envs/xai/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.23.1 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
/home/smarton/anaconda3/envs/xai/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.23.1 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


## Logistic Regression

### Sampled Data

#### basic

In [7]:

data = pump_data_replica.sample(n=100_000)

X_data = data.drop(['state'], axis=1).values
y_data = data[['state']].values

X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.25, random_state=42)           

In [8]:
y_train_rf_pred = random_forest_model.predict(X_train)
y_test_rf_pred = random_forest_model.predict(X_test)

In [9]:
clf = LogisticRegression(random_state=0).fit(X_train, y_train_rf_pred)

preds = clf.predict(X_test)
f1_fidelity = f1_score(y_test_rf_pred, preds)
print('Fidelity (F1 Score) Logistic Regression - Random Forest Model:', f1_fidelity)
acc_fidelity = accuracy_score(y_test_rf_pred, preds)
print('Fidelity (Accuracy) Logistic Regression - Random Forest Model:', acc_fidelity)

f1_performance = f1_score(y_test, preds)
print('Performance (F1 Score) Logistic Regression:', f1_performance)
acc_performance = accuracy_score(y_test, preds)
print('Performance (Accuracy) Logistic Regression:', acc_performance)

Fidelity (F1 Score) Logistic Regression - Random Forest Model: 0.9260428022261747
Fidelity (Accuracy) Logistic Regression - Random Forest Model: 0.94844
Performance (F1 Score) Logistic Regression: 0.8886290232765341
Performance (Accuracy) Logistic Regression: 0.92


In [10]:
clf = LogisticRegression(random_state=0).fit(X_train, y_train)

preds = clf.predict(X_test)
f1_fidelity = f1_score(y_test_rf_pred, preds)
print('Fidelity (F1 Score) Logistic Regression - Random Forest Model:', f1_fidelity)
acc_fidelity = accuracy_score(y_test_rf_pred, preds)
print('Fidelity (Accuracy) Logistic Regression - Random Forest Model:', acc_fidelity)

f1_performance = f1_score(y_test, preds)
print('Performance (F1 Score) Logistic Regression:', f1_performance)
acc_performance = accuracy_score(y_test, preds)
print('Performance (Accuracy) Logistic Regression:', acc_performance)

/home/smarton/anaconda3/envs/xai/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


Fidelity (F1 Score) Logistic Regression - Random Forest Model: 0.9258286289195686
Fidelity (Accuracy) Logistic Regression - Random Forest Model: 0.94692
Performance (F1 Score) Logistic Regression: 0.8930510314875136
Performance (Accuracy) Logistic Regression: 0.9212


#### polynomial features

In [11]:
poly = PolynomialFeatures(degree = 2, interaction_only=False, include_bias=False)
poly.fit(X_train)
X_train_poly = poly.transform(X_train)
X_test_poly = poly.transform(X_test)

clf = LogisticRegression(random_state=0, max_iter=1000).fit(X_train_poly, y_train_rf_pred)

preds = clf.predict(X_test_poly)
f1_fidelity = f1_score(y_test_rf_pred, preds)
print('Fidelity (F1 Score) Logistic Regressionn Polynomial Features - Random Forest Model:', f1_fidelity)
acc_fidelity = accuracy_score(y_test_rf_pred, preds)
print('Fidelity (Accuracy) Logistic Regressionn Polynomial Features - Random Forest Model:', acc_fidelity)

f1_performance = f1_score(y_test, preds)
print('Performance (F1 Score) Logistic Regressionn Polynomial Features:', f1_performance)
acc_performance = accuracy_score(y_test, preds)
print('Performance (Accuracy) Logistic Regressionn Polynomial Features:', acc_performance)

Fidelity (F1 Score) Logistic Regressionn Polynomial Features - Random Forest Model: 0.939042939042939
Fidelity (Accuracy) Logistic Regressionn Polynomial Features - Random Forest Model: 0.95832
Performance (F1 Score) Logistic Regressionn Polynomial Features: 0.8643250297906147
Performance (Accuracy) Logistic Regressionn Polynomial Features: 0.90436


/home/smarton/anaconda3/envs/xai/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [12]:
poly = PolynomialFeatures(degree = 2, interaction_only=False, include_bias=False)
poly.fit(X_train)
X_train_poly = poly.transform(X_train)
X_test_poly = poly.transform(X_test)

clf = LogisticRegression(random_state=0, max_iter=1000).fit(X_train_poly, y_train)


preds = clf.predict(X_test_poly)
f1_fidelity = f1_score(y_test_rf_pred, preds)
print('Fidelity (F1 Score) Logistic Regressionn Polynomial Features - Random Forest Model:', f1_fidelity)
acc_fidelity = accuracy_score(y_test_rf_pred, preds)
print('Fidelity (Accuracy) Logistic Regressionn Polynomial Features - Random Forest Model:', acc_fidelity)

f1_performance = f1_score(y_test, preds)
print('Performance (F1 Score) Logistic Regressionn Polynomial Features:', f1_performance)
acc_performance = accuracy_score(y_test, preds)
print('Performance (Accuracy) Logistic Regressionn Polynomial Features:', acc_performance)

/home/smarton/anaconda3/envs/xai/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


Fidelity (F1 Score) Logistic Regressionn Polynomial Features - Random Forest Model: 0.9125980195752962
Fidelity (Accuracy) Logistic Regressionn Polynomial Features - Random Forest Model: 0.93892
Performance (F1 Score) Logistic Regressionn Polynomial Features: 0.8732222222222221
Performance (Accuracy) Logistic Regressionn Polynomial Features: 0.90872


#### function

In [13]:
preds = np.clip(np.round(10*X_test[:,0]), 0, 1)

f1_fidelity = f1_score(y_test_rf_pred, preds)
print('Fidelity (F1 Score) 2*X0 - Random Forest Model:', f1_fidelity)
acc_fidelity = accuracy_score(y_test_rf_pred, preds)
print('Fidelity (Accuracy) 2*X0 - Random Forest Model:', acc_fidelity)

f1_performance = f1_score(y_test, preds)
print('Performance (F1 Score) 2*X0:', f1_performance)
acc_performance = accuracy_score(y_test, preds)
print('Performance (Accuracy) 2*X0:', acc_performance)

Fidelity (F1 Score) 2*X0 - Random Forest Model: 0.9132092915751762
Fidelity (Accuracy) 2*X0 - Random Forest Model: 0.93992
Performance (F1 Score) 2*X0: 0.8823100644799551
Performance (Accuracy) 2*X0: 0.91604


#### random forest

In [14]:
y_test_random_forest = random_forest_model.predict(X_test)
accuracy_random_forest = accuracy_score(y_test, y_test_random_forest)
print('Accuracy Random Forest: '+ str(accuracy_random_forest))

Accuracy Random Forest: 0.90772


### Complete Data

In [15]:
data = pump_data_replica

X_data = data.drop(['state'], axis=1).values
y_data = data[['state']].values

X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.25, random_state=42)           

In [16]:
y_train_rf_pred = random_forest_model.predict(X_train)
y_test_rf_pred = random_forest_model.predict(X_test)

#### basic

In [17]:
clf = LogisticRegression(random_state=0).fit(X_train, y_train_rf_pred)

preds = clf.predict(X_test)
f1_fidelity = f1_score(y_test_rf_pred, preds)
print('Fidelity (F1 Score) Logistic Regression - Random Forest Model:', f1_fidelity)
acc_fidelity = accuracy_score(y_test_rf_pred, preds)
print('Fidelity (Accuracy) Logistic Regression - Random Forest Model:', acc_fidelity)

f1_performance = f1_score(y_test, preds)
print('Performance (F1 Score) Logistic Regression:', f1_performance)
acc_performance = accuracy_score(y_test, preds)
print('Performance (Accuracy) Logistic Regression:', acc_performance)

Fidelity (F1 Score) Logistic Regression - Random Forest Model: 0.9257879278972286
Fidelity (Accuracy) Logistic Regression - Random Forest Model: 0.9488835999060538
Performance (F1 Score) Logistic Regression: 0.8871270142603659
Performance (Accuracy) Logistic Regression: 0.9199551010945587


In [18]:
clf = LogisticRegression(random_state=0).fit(X_train, y_train)

preds = clf.predict(X_test)
f1_fidelity = f1_score(y_test_rf_pred, preds)
print('Fidelity (F1 Score) Logistic Regression - Random Forest Model:', f1_fidelity)
acc_fidelity = accuracy_score(y_test_rf_pred, preds)
print('Fidelity (Accuracy) Logistic Regression - Random Forest Model:', acc_fidelity)

f1_performance = f1_score(y_test, preds)
print('Performance (F1 Score) Logistic Regression:', f1_performance)
acc_performance = accuracy_score(y_test, preds)
print('Performance (Accuracy) Logistic Regression:', acc_performance)

/home/smarton/anaconda3/envs/xai/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


Fidelity (F1 Score) Logistic Regression - Random Forest Model: 0.9272413647233945
Fidelity (Accuracy) Logistic Regression - Random Forest Model: 0.9486244744521862
Performance (F1 Score) Logistic Regression: 0.8924138714058995
Performance (Accuracy) Logistic Regression: 0.9218408490581343


#### polynomial features

In [19]:
poly = PolynomialFeatures(degree = 2, interaction_only=False, include_bias=False)
poly.fit(X_train)
X_train_poly = poly.transform(X_train)
X_test_poly = poly.transform(X_test)

clf = LogisticRegression(random_state=0, max_iter=1000).fit(X_train_poly, y_train_rf_pred)

preds = clf.predict(X_test_poly)
f1_fidelity = f1_score(y_test_rf_pred, preds)
print('Fidelity (F1 Score) Logistic Regression Polynomial Features - Random Forest Model:', f1_fidelity)
acc_fidelity = accuracy_score(y_test_rf_pred, preds)
print('Fidelity (Accuracy) Logistic Regressionn Polynomial Features - Random Forest Model:', acc_fidelity)

f1_performance = f1_score(y_test, preds)
print('Performance (F1 Score) Logistic Regressionn Polynomial Features:', f1_performance)
acc_performance = accuracy_score(y_test, preds)
print('Performance (Accuracy) Logistic Regressionn Polynomial Features:', acc_performance)

/home/smarton/anaconda3/envs/xai/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Fidelity (F1 Score) Logistic Regression Polynomial Features - Random Forest Model: 0.9369749049259039
Fidelity (Accuracy) Logistic Regressionn Polynomial Features - Random Forest Model: 0.9574079024317612
Performance (F1 Score) Logistic Regressionn Polynomial Features: 0.8645714020622464
Performance (Accuracy) Logistic Regressionn Polynomial Features: 0.9057191216541696


In [20]:
poly = PolynomialFeatures(degree = 2, interaction_only=False, include_bias=False)
poly.fit(X_train)
X_train_poly = poly.transform(X_train)
X_test_poly = poly.transform(X_test)

clf = LogisticRegression(random_state=0, max_iter=1000).fit(X_train_poly, y_train)

preds = clf.predict(X_test_poly)
f1_fidelity = f1_score(y_test_rf_pred, preds)
print('Fidelity (F1 Score) Logistic Regressionn Polynomial Features - Random Forest Model:', f1_fidelity)
acc_fidelity = accuracy_score(y_test_rf_pred, preds)
print('Fidelity (Accuracy) Logistic Regressionn Polynomial Features - Random Forest Model:', acc_fidelity)

f1_performance = f1_score(y_test, preds)
print('Performance (F1 Score) Logistic Regressionn Polynomial Features:', f1_performance)
acc_performance = accuracy_score(y_test, preds)
print('Performance (Accuracy) Logistic Regressionn Polynomial Features:', acc_performance)

/home/smarton/anaconda3/envs/xai/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/home/smarton/anaconda3/envs/xai/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Fidelity (F1 Score) Logistic Regressionn Polynomial Features - Random Forest Model: 0.9259074252626986
Fidelity (Accuracy) Logistic Regressionn Polynomial Features - Random Forest Model: 0.9477779575364796
Performance (F1 Score) Logistic Regressionn Polynomial Features: 0.8756692947464401
Performance (Accuracy) Logistic Regressionn Polynomial Features: 0.9098363203518405


In [24]:
poly = PolynomialFeatures(degree = 3, interaction_only=False, include_bias=False)
poly.fit(X_train)
X_train_poly = poly.transform(X_train)
X_test_poly = poly.transform(X_test)

clf = LogisticRegression(random_state=0, max_iter=10000).fit(X_train_poly, y_train_rf_pred)

preds = clf.predict(X_test_poly)
f1_fidelity = f1_score(y_test_rf_pred, preds)
print('Fidelity (F1 Score) Logistic Regression Polynomial Features - Random Forest Model:', f1_fidelity)
acc_fidelity = accuracy_score(y_test_rf_pred, preds)
print('Fidelity (Accuracy) Logistic Regressionn Polynomial Features - Random Forest Model:', acc_fidelity)

f1_performance = f1_score(y_test, preds)
print('Performance (F1 Score) Logistic Regressionn Polynomial Features:', f1_performance)
acc_performance = accuracy_score(y_test, preds)
print('Performance (Accuracy) Logistic Regressionn Polynomial Features:', acc_performance)

Fidelity (F1 Score) Logistic Regression Polynomial Features - Random Forest Model: 0.9098142031292472
Fidelity (Accuracy) Logistic Regressionn Polynomial Features - Random Forest Model: 0.9412928464847714
Performance (F1 Score) Logistic Regressionn Polynomial Features: 0.8267580452381749
Performance (Accuracy) Logistic Regressionn Polynomial Features: 0.8836977035480476


In [25]:
poly = PolynomialFeatures(degree = 3, interaction_only=False, include_bias=False)
poly.fit(X_train)
X_train_poly = poly.transform(X_train)
X_test_poly = poly.transform(X_test)

clf = LogisticRegression(random_state=0, max_iter=10000).fit(X_train_poly, y_train)

preds = clf.predict(X_test_poly)
f1_fidelity = f1_score(y_test_rf_pred, preds)
print('Fidelity (F1 Score) Logistic Regressionn Polynomial Features - Random Forest Model:', f1_fidelity)
acc_fidelity = accuracy_score(y_test_rf_pred, preds)
print('Fidelity (Accuracy) Logistic Regressionn Polynomial Features - Random Forest Model:', acc_fidelity)

f1_performance = f1_score(y_test, preds)
print('Performance (F1 Score) Logistic Regressionn Polynomial Features:', f1_performance)
acc_performance = accuracy_score(y_test, preds)
print('Performance (Accuracy) Logistic Regressionn Polynomial Features:', acc_performance)

/home/smarton/anaconda3/envs/xai/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


Fidelity (F1 Score) Logistic Regressionn Polynomial Features - Random Forest Model: 0.8584862553370056
Fidelity (Accuracy) Logistic Regressionn Polynomial Features - Random Forest Model: 0.9033597002453732
Performance (F1 Score) Logistic Regressionn Polynomial Features: 0.799903671209226
Performance (Accuracy) Logistic Regressionn Polynomial Features: 0.8592772901633461


#### function

In [21]:
preds = np.clip(np.round(10*X_test[:,0]), 0, 1)

f1_fidelity = f1_score(y_test_rf_pred, preds)
print('Fidelity (F1 Score) 2*X0 - Random Forest Model:', f1_fidelity)
acc_fidelity = accuracy_score(y_test_rf_pred, preds)
print('Fidelity (Accuracy) 2*X0 - Random Forest Model:', acc_fidelity)

f1_performance = f1_score(y_test, preds)
print('Performance (F1 Score) 2*X0:', f1_performance)
acc_performance = accuracy_score(y_test, preds)
print('Performance (Accuracy) 2*X0:', acc_performance)

Fidelity (F1 Score) 2*X0 - Random Forest Model: 0.9126362215656276
Fidelity (Accuracy) 2*X0 - Random Forest Model: 0.9403104744452115
Performance (F1 Score) 2*X0: 0.8796815984039408
Performance (Accuracy) 2*X0: 0.9153439113326979


#### random forest

In [22]:
y_test_random_forest = random_forest_model.predict(X_test)
accuracy_random_forest = accuracy_score(y_test, y_test_random_forest)
print('Accuracy Random Forest: '+ str(accuracy_random_forest))

Accuracy Random Forest: 0.9078300312906494


In [23]:
z

NameError: name 'z' is not defined